# Clustering

## Approches
Parmi toutes les grands type d'approche des problèmes de clustering, on peut mentionner les approches:
* *Model-based* comme les *Gaussian mixture models*
* *Distance-based partionning* comme les algorithmes *k-means* ou *k-medoids*
* *Distance-based hierarchical* au sein desquels on distingue les méthodes agglomératives (*bottom-up*) et divisives (*top-down*).
* *Density based* comme les algorithmes DBSCAN ou OPTICS.
* Avec réduction de dimension comme le *spectral clustering*, la *non-negative matrix factorization*.

https://fr.slideshare.net/Krish_ver2/32-partitioning-methods
https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a

## Approches *density-based*
### DBSCAN (*Density-Based Spatial Clustering of Applications with Noise*)
L'idée de DBSCAN est simple et consiste à découvrir les regroupements de points satisfaisant une condition de densité minimale, ces regroupements étant séparés par des zones de densité inférieure. La densité spatiale minimale (points par unité de volume) est définie comme un nombre minimal de points `min_samples` contenues dans le volume correspondant à une boule de rayon `epsilon`. On appelle core point un point dont le voisinage respecte la condition de densité minimale (la boule de rayon `epsilon` centrée sur le point contient au moins `min_samples`, son centre inclus). Un core point consitue le germe d'un cluster qui est ensuite étendu à partir de celui-ci selon une procédure récursive:
* Tous les voisins d'un core point sont ajoutés au cluster assigné à celui-ci (Maximalité)
* On vérifie si chacun des voisins sont aussi des core points, et la procédure est répétée pour chacun de ceux validant la core point condition jusqu'à ce qu'on ne puisse plus ajouter de points au cluster.
Quand on ne peut plus ajouter de points au cluster, on amorce à nouveau l'algorithme sur un point non encore assigné. Les points ne pouvant à l'issue de la procédure, être associés à un cluster sont désignés comme bruit (noise)/outliers.

DBSCAN (ainsi que OPTICS) introduisent différentes notions "d'atteignabilité" (*reachability*) d'un point qui permmettent de caractériser différents types de points et de donner un définition formelle d'un cluster au sens de DBSCAN. De la notion la plus forte à la plus faible: 
* *direct density reachability* : Un point B est dit *directly density reachable* depuis un point A s'il se trouve dans le voisignage direct de A (à une distance inférieure à `epsilon`) et que A est un *core point*. Cette notion n'est pas symétrique en général (si B n'est lui même pas un *core point*). Elle est symétrique si A et B sont des *core points*.
* *density reachability*: Un point B est dit *density reachable* depuis un point A s'il existe une chaîne de points permettant de relier A à B dans laquelle le point suivant est toujours *directly density reachable* depuis le point précédent. Là encore, la notion n'est pas symétrique en général et l'est si A et B sont des *core points*.
* *density-connected*: Un point A est dit *density connected* à un point B s'il existe au moins un point C tels que A et B sont *directly density reachable* depuis C. Cette relation est symétrique.

Remarque: Ces notions sont attachés à une densité minimale qu'on se donne (couple (`epsilon`, `n_samples`)), deux points A et B peuvent être par exemple *density connected* pour une densité X mais pas pour une densité Y.

On définit ainsi précisément les différents types d'objets mis en évidence par DBSCAN:
* Un *density-based cluster* se définit par deux propriétés:
    * Maximalité (règle pour construire le cluster): Si A appartient au cluster et B est *directly density reachable* depuis A, alors B appartient au cluster.
    * Connectivité: Quels que soient A et B appartenant au même cluster, A et B sont *density-connected*. Dit autrement, un cluster se définit comme l'ensemble des points *density reachable* depuis un *core point* arbitrairement choisi.
* Un point est appelé *core point* si sont voisinage satisfait la condition de densité minimale.
* Un point est appelé *border point* s'il est *density reachable* mais ne satisfait pas la condition de densité minimale (aucun point n'est *directly density reachable* depuis un *border point*). Ces deux propriétés se rencontrent notamment sur les bords des clusters d'où le nom.
* Un point est assimilé à du bruit (*noise*) s'il n'appartient à aucun *density-based cluster*, ce qui signifie qu'il n'existe pas de *core point* à partir du- ou desquels ce point est *directly density reachable*.

```python
def DBSCAN(dataset, epsilon, n_samples):
    C = 0 # Cluster counter
    for point in dataset:
        if point.label is None:
            neighbors = get_neighborhood(point, dataset, epsilon)
            if is_core_point(neighbors, n_samples): # We start a new cluster
                point.label = C
                label_neighborhood(C, point, dataset, epsilon, n_samples)
            else: # We label as noise
                point.label = -1


def is_core_point(neighbors, n_samples):
    return len(neighbors) < n_samples


def label_neighborhood(C, point, dataset, epsilon, n_samples):
    neighbors = get_neighborhood(point, dataset, epsilon)
    for neighbor in neighbors: # Includes `point` but it has already been labelled
        if neighbor.label = -1: # If already labelled as noise we assign to current cluster (border point)
            neighbor.label = C
        elif neighbor.label is not None: # If already assigned to a cluster, we leave as is
            continue
        else: # If core point we keep exploring, if not the point will be assigned later
            if is_core_point(neighbor, dataset, epsilon, n_samples):
                neighbor.label = C
                label_neighborhood(C, neighbor, dataset, epsilon, n_samples)
            else:
                continue

```

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
illustration
principaux papiers
http://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/notes_slides/revisitofrevisitDBSCAN.pdf
clair mais pas le papier principal.

Avantages/Cas d'usage
* Clusters de taille inégale 
* Clusters n'ayant pas forcément une géométrie simple (notamment convexe)
* Contrairement aux *partionning methods*, n'a pas le désavantage d'imposer de spécifier un nombre de clusters à priori.  
Inconvénients 
* Etant *distanced-based* les méthodes *density-based* sont sujettes à la curse of dimensionnality
* Stabilité/déterminisme: Il semble que DBSCAN n'est pas totalement déterministe (sans doute lié au fait qu'il est *greedy*) pour les *border points* dont l'assignation dépend de l'ordre dans lequel l'algorithme traite les points (mais pas de référence à une seed dans l'algo sklearn ?) et donc de l'état initial. L'algorithme est déterministe pour les core points mais pas pour les border points (provient du fait qu'une fois assigné à un cluster, un point n'est plus réassigné contrairement à d'autres algorithmes comme le k-means et l'assignation d'un border point à un cluster dépend du chemin par lequel on y arrive). Etant donné l'ordre des données, DBSCAN est parfaitement déterministe. Si on permutte on peut aboutir à des résultats très légèrement différents liés à l'assignation des border points.
* Le principal inconvénient de DBSCAN est qu'il peut mal performer s'il existe d'importantes différences de densité (mais pas forcément de taille) entre les clusters. DBSCAN n'utilise en effet par construction qu'un unique seuil de densité minimale pour l'ensemble du *dataset*. En cas d'importantes différences de densité, cet unique seuil peut ne pas être suffisant pour détecter tous les clusters. Suivant les valeurs de `epsilon`, `n_samples` choisies on peut par exemple finir avec des clusters entiers labelisés comme bruit (si seuil trop élevé, on arrive pas à capturer les clusters de densité faibles) ou encore des cluster qui se trouvent regroupés en un seul (si seuil trop faible pour capturer les clusters de densité faibles, les clusters denses mais proches peuvent se faire fusionner). C'est à ce principal désavantage que viennent répondre des méthodes comme l'algorithme OPTICS. 

Prédiction

### OPTICS 
https://www.dbs.ifi.lmu.de/Publikationen/Papers/OPTICS.pdf
http://ceur-ws.org/Vol-2191/paper37.pdf
https://slideplayer.com/slide/4239131/
https://www.slideshare.net/rpiryani/optics-ordering-points-to-identify-the-clustering-structure

OPTICS produit pour un `n_samples` fixé une liste correspondant à l'ensemble des points du dataset indexés dans l'ordre dans lequel ils ont été traités (appelé *cluster order*). Une *reachability distance* a été calculée pour chacun de ces points. Cette distance correspond au `epsilon` le plus faible pour lequel le point peut être *directly density reachable* depuis le *core point* par rapport auquel la distance a été calculée. La *reachability distance* peut:
* Être indéfinie (inifiniment élevée) dans le cas des points servant à initialiser l'algorithme.
* Être égale à la *core distance* (cf. ci-dessous) du *core point* par rapport auquel elle est calculée si le point fait partie des `n_samples` points les plus proches du *core point*. En effet si on prenait la distance du point au *core point* comme *reachability distance*, alors prendre `epsilon` égal à cette distance ne permettrait pas au *core point* d'être un *core point* pour la valeur de `n_samples` donnée.
* Être simplement égale à la distance au *core point* par rapport auquel elle est calculée si celle-ci est supérieure à la *core distance* de celui-ci.

Remarque: La *core distance* d'un point correspond à la distance $\epsilon$ minimale pour que ce point puisse être un *core point* pour le nombre `n_samples` qu'on se donne. C'est le rayon minimal à donner à la boule définissant le voisinage du point pour que celui-ci inclut `n_samples`.

Que signifie pour un point d'avoir une *reachability distance* de $d$ ? Cela signifie que ce point est *directly density reachable* depuis le point le précédant pour toute densité $(\epsilon, n\_samples)$ avec $\epsilon \geq d$ ou de façon équivalente, pour toute densité égale à ou plus faible que $(\epsilon, n\_samples)$. La *reachability distance* variant de façon monotone par bloc sur la séquence construite et retournée par OPTICS, on peut en déduire facilement pour une valeur de densité/$\epsilon$ donnée une ou plusieurs séquences de points *density-connected* pour cette densité, c'est à dire des clusters. A l'inverse, si la *reachability distance* d'un point est supérieure au $\epsilon$ qu'on se donne, cela signifie qu'il n'existe aucun *core point* pour lequel ce point est *directly density reachable* (et est donc labellisé comme bruit). 

OPTICS ne retourne pas des clusters (une liste de labels) mais une réindexation des points du dataset pour chacun desquels a été calculé une *core distance* et une *reachability distance*. Les cluster peuvent en être extraits de deux principales façons:
* Une méthode (`cluster_method='dbscan'` dans `sklearn`) retournant des résultats identiques à ceux d'un DBSCAN (ou quasiment identiques, quelques *border points* peuvent se retrouver assignés différemment entre les deux méthodes). Dans cette méthode on se donne une densité minimale (un epsilon, argument `eps` dans `sklearn`) pour définir les clusters. La densité minimale étant la même pour l'ensemble du dataset, on se retrouve avec les mêmes écueils que DBSCAN en cas d'importants écarts de densité entre les clusters. L'algorithme d'extraction des clusters est simple et consiste à traverser la liste ordonnée une seule fois. Pour chaque point de la liste ordonnée retournée par OPTICS:
    * Si la *reachability distance* est supérieure à `eps`: 
        * Si le point est un *core point* on démarre un nouveau cluster auquel le point courant est assigné,
        * Sinon le point est labellisé comme bruit (car aucun point n'est *directly density reachable* depuis celui-ci).
    * Si la *reachability distance* est inférieure ou égale à `eps`, le point courant est assigné au cluster courant.
* Une méthode (`cluster_method='xi'` dans `sklearn`) qui permet d'extraire des clusters de densité différentes en se basant sur un autre critère. L'idée est de remarquer qu'un cluster "commence" quand la *reachability distance* chutte brutalement (les points sont soudainement plus proches de leur prédécesseur) et se terminent quand elle remonte brutalement (sont soudainement plus éloignés de leur prédécesseur). On va donc se donner un paramètre $\xi \in ]0,1[$ permettant de formaliser cette notion de remonter/descendre brutalement. Par exemple, "diminuer brutalement" sera défini par une *reachability distance* au plus égale à $(1-\xi)$ celle de son prédécesseur. Les auteurs d'OPTICS définissent pour délimiter les clusters des zones où les *reachability distance* augmentent ou diminuent rapidement appelées $\xi$-*upward (resp. downward) area*. Une $\xi$-*upward area* se définie comme un ensemble de points consécutifs satisfaisant:
* Le premier et le dernier point de la $\xi$-*upward area* sont $\xi$-*steep upward* (un point est au moins $\xi$% plus bas que son successeur).
* Quel que soit le point de la $\xi$-*upward area*, sa *reachability distance* est toujours au moins égale à celle de son prédécesseur.
* Une $\xi$-*upward area* ne peut pas comporter plus de `n_samples` points qui ne sont pas $\xi$-*steep upward*.
On en déduit alors la définition d'un $\xi$-cluster qu'on peut résumer commme un ensemble de points encadré d'une $\xi$-*steep upward area* et d'une $\xi$-*steep downward area*: 
* Le premier (resp. dernier) point du cluster appartient à une $\xi$-*steep downward (resp. upward) area*.
* Les *reachability distances* de tous les points d'un $\xi$-cluster doivent être au moins $\xi$% moins élevée que la *reachability distance* du premier point du cluster et que celle du point suivant le dernier point du cluster.
* Pour être cohérent avec la définition de la *core condition*, un $\xi$-cluster doit consister en au moins `n_samples` points.
* Définition permettant de trouver le premier et le dernier point d'un $\xi$-cluster: on veut globalement que les *reachability distances* du premier point et du point situé après le dernier point du cluster soient proches (la plus grande ne doit pas dépasser l'autre de plus de $\xi$%). Cette condition permet notamment de de mettre en évidence d'éventuelles hiérarchies de cluster (clusters plus denses situés à l'intérieur de clusters moins denses). 

OPTICS permet de mettre en évidence des hiérarchies de cluster car l'algorithme construit la liste de points en prenant toujours le candidat avec la *reachability distance* la plus faible. Cela garantit que quand l'algorithme s'attaque à un cluster plus dense situé à l'intérieur d'un cluster moins dense qu'il a déjà partiellement traité, de finir d'ajouter à la liste les points du cluster le plus dense avant de finir le cluster le moins dense. Ainsi, lorsqu'on regarde le graphique des *reachability distances*, on va les voir diminuer deux fois puis remonter deux fois mettant ainsi en évidence l'existence de clusters emboités.


```python
def OPTICS(dataset, epsilon, n_samples):
    cluster_order = []
    for point in dataset:
        if point not in cluster_order:
            neighbors = get_neighborhood(point, dataset, epsilon)
            point.reachability_distance = None # Undefined reachability distance
            point.core_distance = get_core_distance(neighbors, point, n_samples)
            cluster_order.append(point)
            if point.core_distance is not None: # If core point 
                sorted_stack = update_stack(Stack(), neighbors, center_point, cluster_order) # Init sorted stack
                processed_points = process_neighborhood(sorted_stack, [], dataset, epsilon, n_samples)
                cluster_order.extend(processed_points)

    return cluster_order


def process_neighborhood(sorted_stack, processed_points, dataset, epsilon, n_samples):
    if not sorted_stack.empty():
        current_point = sorted_stack.pop(0)
        current_point_neighbors = get_neighborhood(current_point, dataset, epsilon)
        current_point.core_distance = get_core_distance(current_point_neighbors, current_point, n_samples)
        processed_points.append(current_point)
        if current_point.core_distance is not None: # If core point
            sorted_stack = update_stack(sorted_stack, current_point_neighbors, current_point, processed_points)
        process_neighborhood(sorted_stack, processed_points, dataset, epsilon, n_samples)

    return processed_points


def update_stack(sorted_stack, neighbors, center_point, processed_points):
    for neighbor in neighbors:
        if neighbor not in processed_points:
            new_reachability = max(center_point.core_distance, dist(center_point, neighbor)) 
            # Notice that reachability depends on a center point
            if neighbor in sorted_stack: # If point already in stack, reachability distance updated only if smaller
                new_reachability = min(new_reachability, neighbor.reachability_distance)
            neighbor.reachability_distance = new_reachability
            sorted_stack.upsert(neighbor) # insert new or update if already existing
            
    return sorted_stack.sort(by='reachability', ascending=True)
```
On remarque que la récupération du voisinage va récupérer tous les points jusqu'à une distance $\epsilon_{max}$ appelée *generating distance*. Dans `sklearn` cette distance maximale est contrôlée par le paramètre `max_eps` égal à `np.inf` par défaut. Réduire la valeur de la *generating distance* peut permettre d'aboutir à des temps de calcul réduits sans pour autant impacter la structure découverte par OPTICS (jusqu'à une certaine limite). On remarque que la *generating distance* pose une borne supérieure aux *core distances* (au delà elles sont considérées comme indéfinies, comme lorsque le nombre de voisins est inférieur à `n_samples` - i.e. fixé à `np.inf`). On rappelle que le fait d'avoir une *core distance* définie trace la frontière entre les points pouvant former la base d'un cluster, les *core points* et ceux qui seront assimiler à du bruit (*noise points*) sauf si *directly density-reachable* depuis un *core point* (*border point*). 

Un des avantages de OPTICS est aussi que la structure des *reachability distance* bouge relativement peu pour une gamme de paramètres (`n_samples`, `max_eps`) assez large. Le paramétrage de l'algorithme est donc relativement peu exigeant puisque ses résultats sont relativement stables sur une plage raisonnable de valeurs des paramètres. 

Comme DBSCAN, OPTICS en mode xi a l’avantage de pouvoir séparer le bruit des autres clusters? bruit = tt ce qui n’est pas rentré dans la définition du cluster ?

Avantage par rapport au single-linkage clustering (cf. début du papier).
https://en.wikipedia.org/wiki/Single-linkage_clustering

## LOF
Le LOF consiste à comparer via un score, la densité locale d'un point avec la densité locale moyenne de ses k plus proches voisins. Les définitions de densité font toujours référence à une notion de masse et une notion de volume. Dans le cas du LOF, la densité locale d'un point se définie comme la taille de son k-voisinage : la masse incluse dans le voisinage est toujours à peu près la même d'un point à l'autre (~k points), plus on a dû aller loin pour aller cherche un même nombre de points, plus la densité est faible. On définit la densité locale d'un point A comme l'inverse de la "distance" moyenne de ce point depuis chacun des membres de ses k voisinage. La notion de distance utilisée ici est particulière et appelée reachability distance depuis un point: elle correspond à la distance usuelle mais avec un plancher.  Le plancher correspond à la core distance du point par rapport à laquelle la reachability distance est calculée (ce qui fait qu'elle n'est pas symétrique et donc pas vraiment une distance). L'argument pour mettre un tel plancher (dépendant de k et du point) est principalement un meilleure stabilité de l'algorithme (cela évite aux densités de trop diverger dans les zones très denses). On aurait aussi pu prendre pour la densité la k-distance au point A, là encore la moyenne permet d'apporter de la stabilité : imaginons que les k-1 voisins sont relativement près mais le ke assez loin: la k-distance sera élevée bien que la distance moyenne est faible et donc la densité locale assez élevée. On calcule ainsi les densités locales du point courant et celle des membres de son k voisinage. On les moyenne les densités locales de voisins et la rapporte à celle du point courant. Le fait de se reposer sur un ratio fait partie de l'idée d'étudier des différences relatives de densité au niveau local: le LOF peut ainsi s'accommoder de différentes densités au sein du dataset. Le score obtenu sera donc supérieur à 1 si la densité locale du point courant est plus faible que celle de ses voisins (ex: la majeure partie des voisins est un peu éloignée et de densité locale faible - appartenant à un cluster - alors que la densité locale du point courant est faible), proche de 1 si elle en est proche et inférieur à 1 si elle est plus élevée. Dit autrement, la densité moyenne locale des voisins est "score" fois plus élevée que la densité moyenne locale du point courant. Le LOF donne ainsi une mesure de "l'outlierness", à nous de fixer le seuil à partir duquel un point doit être considéré comme outlier. 

En choisissant délibérément de prendre une échelle locale, le LOF évite les écueils des approches globales où un seul seuil de densité minimal est fixé pour l'ensemble des données. k est le paramètre permettant de régler le degré de localité. Plus k est élevé, plus on va chercher des voisins éloignés avec lesquels il peut être de moins en moins pertinent de comparer sa densité car il est de moins en moins pertinent de parler de voisins. A contrario plus k est faible, plus le voisinage risque de ne pas être "représentatif" (Ex: pour un outlier dont les points proches sont aussi de densité locale faible mais aller plus loin aurait montré que l'essentiel du voisinage est de densité élevée, le score ne sera pas aussi élevé que si on était allé plus loin. A l'inverse, un point appartenant à un petit pâté isolé peu faire croire à un petit cluster alors qu'en allant plus loin on aurait pu voir qu'il s'agit d'un "groupe d'outliers") et moins le résultat apparaîtra stable (au sens où il semblera varier beaucoup quand on augmente k).

En résumé, trois choses importent dans l'appréciation de "l'outlierness" d'un point A:
* Le nombre de voisins qu'on choisit de considérer k
* La distance (précisons au sens de la reachability distance) de ces k voisins au point A (définit la densité locale de A)
* La densité locale de chacun de ces voisins (définit la densité locale moyenne à laquelle on compare celle de A).

En prédiction : comme kNN ? approche prototype ? comment on fait ?

### GMM
https://jakevdp.github.io/PythonDataScienceHandbook/05.12-gaussian-mixtures.html#How-many-components?
Bishop: http://www.rmki.kfki.hu/~banmi/elte/bishop_em.pdf
http://www.cse.psu.edu/~rtc12/CSE586/lectures/EMLectureFeb3.pdf

### Autres
 Silhouette
 métriques en clustering / sélection de K
 https://developers.google.com/machine-learning/clustering/algorithm/advantages-disadvantages


### Outlier detection
https://github.com/SeldonIO/alibi-detect